In [101]:
#get trip type href
import requests
import re
import time
from bs4 import BeautifulSoup as bs

res = requests.get('http://gabriel.com.tw/chinese/01_journey/00_overview.php?MID=3', 'lxml')
res.encoding = 'utf-8'
soup = bs(res.text)

In [111]:
JPNList = []
for ele in soup.select('.current a'):
    #get all japan package project no.
    if re.search('MID=(.*)',ele['href']) != None:
        #3=日本心漫遊, 7=蜜月樂園, 2=日本極上宿, 4=日本鐵道
        if int(re.search('MID=(.*)',ele['href']).group(1)) in [2,3,4,7]:
            for projIndex in re.search('MID=(.*)',ele['href']).group(1):
                JPNList.append(projIndex) 

In [158]:
projectList = []
for projIndex in JPNList:
    fullDic = {}
    fullDic['agency'] = 'gabriel'
    #get each package project info.
    resPack = requests.get('http://gabriel.com.tw/chinese/01_journey/00_overview.php?MID={}'.format(projIndex), 'lxml')
    resPack.encoding = 'utf-8'
    soupPack = bs(resPack.text)
    for proj in soupPack.select('.tour_over_wrap'):
        fullDic['title'] = proj.select('.tourcountry')[0].text+'-'+proj.select('.tour_over_wrapL_des')[0].text
        fullDic['href'] = 'http://gabriel.com.tw/chinese/01_journey/'+proj.select('.tour_over_wrapL_des a')[0]['href']
        fullDic['prodNo'] = re.search('AID=(.*)',proj.select('.tour_over_wrapL_des a')[0]['href']).group(1)
        #get each project info.
        resProj = requests.get(fullDic['href'], 'lxml')
        resProj.encoding = 'utf-8'
        soupProj = bs(resProj.text)
        tourDic = {}
        #it is different of projIndex = 2 i.e. 日本極上宿
        #get tour info.
        if projIndex == '2':
            i=1
            for tour in soupProj.select('.detail_edit'):
                day = 'day{}'.format(i)
                for tourA in tour.select('table tbody tr td table tbody tr td table tbody tr td table tbody tr td'):
                    if len(tourA.text)>2:
                        A = tourA.text.lstrip().strip('\n')
                for tourB in tour.select('table tbody p span span'):
                    if len(tourB.text.lstrip().strip('\n'))>10:
                        B = tourB.text.lstrip().strip('\n')
                tourDic[day] = A+','+B
                i = i+1
        else:
            #get tour info.
            try:
                i=1
                for tour in soupProj.select('.detail_edit h2'):
                    day = 'day{}'.format(i)
                    tourDic[day] = tour.text.lstrip().strip('\n')
                    i = i+1
            except AttributeError:
                pass
        tempTourDic = tourDic.copy()
        fullDic['tour'] = tempTourDic
        tempFullDic = fullDic.copy()
        projectList.append(tempFullDic)

3
7
2
桃園│名古屋,搭乘國泰航空商務艙 ~中部國際空港 ~晚餐預訂：螃蟹鐵板燒料理 或 主廚傳統握壽司~住宿名古屋飯店。
名古屋－伊勢,日本設計風列車．島風號 ~與海女相遇．伊勢灣海女小屋燒烤海鮮 ~珠寶藝術．大山玉寶美術館 ~嚴選伊勢湯宿（房間附溫泉風呂）．沉浸伊勢星空夜語
伊勢神宮,睡到自然醒，盡情享受極上湯宿時光 ~おかげ橫丁．江戶風古街．伊勢名物『赤福』 ~日本三大神宮『伊勢神宮』~世界頂級旅宿．安縵系列唯一溫泉度假村．安縵伊沐
伊勢－名古屋,沒有MORNING CALL的早晨 ~安縵伊沐午膳 ~休閒新設計美學．AQUA x IGNIS ~晚餐安排 松阪牛肉料理 或 特色料亭 ~繁華榮町區．名古屋夜未眠
名古屋－台北,日本德川園．迴遊式庭園． ~德川園洋式午膳 或名古屋鰻魚三吃套餐~中部國際空港．江戶風商店街 ~台北
台北│日本．靜岡空港 －40km －靜岡 ,日本．靜岡空港 ~綠茶之鄉．靜岡 
靜岡－50km －掛川－ 60km －日本平,掛川花鳥園．哈利波特信使 ~東海名城．掛川城．木造天守閣 ~庭園料亭．本格
                                    味自慢料理 ~【鐵道之旅－大井川鐵道．SL復古蒸氣火車】~富士山之夜．風景
                                    美術館精品宿「日本平」
日本平－御殿場－熱海,日本平空中纜車．久能山東照宮．駿河灣海景 ~日本最大季外品購物中心．御殿場OUTLET ~熱海建築美學．起雲閣．抹茶品茗  ~住宿極上宿．沉浸專屬私人寧靜．熱海石亭 ~晚間欣賞2016熱海海上花火大會
熱海－鐮倉－東京 或 橫濱,天青酒色．湘南唯一藏元．熊澤酒造 ~下一站幸福．鐵道小旅行．懷舊江之電 ~漫畫「灌籃高手」．湘南海岸 ~千年古都悸動．鎌倉．鶴岡八幡宮~宿の圓舞曲：【A. 橫濱．21未來港 或 B. 東京．璀璨夜未眠】 
東京 －東京國際空港 －台北,早安！TOKYO！~如時間許可（搭乘晚班機）東京趴趴GO．自由逛街購物 ~光伸免稅店 ~東京國際空港．機場商店街 ~台北
台北│日本．東京 ,日本．東京．成田國際空港 ~宿の圓舞曲：【A. 橫濱．21世紀未來港．置地廣場 或 B. 東京．都會霓虹璀璨】 
東京－山梨－河口湖,The Premium Malt's‧三多利武藏

In [159]:
import json
from json import load
from pprint import pprint
with open('gabriel.json', 'w') as f:
     json.dump(projectList, f)
        

In [160]:
with open('gabriel.json') as data_file:    
    data = json.load(data_file)
for ele in data:
    for key in sorted(ele):
        if key == 'tour':
            for tourkey in sorted(ele[key]):
                print tourkey,':',ele[key][tourkey]
        else:
            print key,':',ele[key]

agency : gabriel
href : http://gabriel.com.tw/chinese/01_journey/01_detail.php?AID=506
prodNo : 506
title : 【JFC6】九州．鐵道Q旅行-來去『湯布院』住二晚6日
day1 : 台北 －日本．九州．福岡
旅遊地圖：日本．九州．福岡空港 ~庶民美食．「一蘭拉麵」或「博多だるま」~都會璀璨夜景．數著星星入眠
day2 : 福岡 －柳川 －佐賀
旅遊地圖：太宰府物語：【A. 天滿宮．梅枝餅 或 B. 隈研吾．星巴克】~小京都探訪．柳川扁舟悠遊．輕槳擺渡船歌 ~佐賀阿嬤故鄉：【A. 見返瀑布．紫陽花 或 B. 武雄名園．御船山樂園 或 C. 九重葛之家 或 D. 佐賀城跡＋佐賀錦】
day3 : 佐賀 －日田 －湯布院
旅遊地圖：穿越時光隧道：【A. 薰長酒藏．冰淇淋 或 B. 豆田町．日本丸館】~下一站 幸福．『ゆふいんの森』（日田－由布院）~湯布院．尋寶趣（自費）：【A. 排隊甜點「B-Speak」或 B. SNOOPY茶屋 或 C. 金賞可樂餅 或 D. GOEMON蛋糕】 
day4 : 湯布院 －大分 －湯布院
旅遊地圖：藝術鑑賞．大分縣立美術館 ~旬彩風物詩：【A. 神樂女湖．菖蒲花 或 B. 鶴見岳．空中纜車 或 C. 由布川峽谷 或 D. 海地獄】~午后浪漫時光．咖啡甜點 ~豐後富士．由布院小鎮．金麟湖散策 ~將「心」遺留在『湯布院』
day5 : 湯布院 －福岡
旅遊地圖：幸福MORNING CALL ~人氣秘湯：黑川溫泉．野溪風呂．地藏堂布丁 ~翩翩花舞悸動：【A. くじゅう花公園 或 B. 九重「夢」大吊橋】~如時間許可加贈：【A. 鳥栖 PREMIUM OUTLETS 或 B. 博多運河城】~光伸免稅店 ~福岡夜未眠
day6 : 福岡 －台北
旅遊地圖：早安！FUKUOKA！ ~福岡國際空港  ~台北
agency : gabriel
href : http://gabriel.com.tw/chinese/01_journey/01_detail.php?AID=517
prodNo : 517
title : 【特價】哈利波特．環球影城-三大慶典．京都祇園祭6日
day1 : 台北 －日本．大阪
旅遊地圖：日本．大阪．關西國際空港 ~宵夜